In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import numpy as np
import random
import time
import pygame
import pickle

from tetris.Piece import Piece
from tetris.EnvironmentRendered import TetrisEnvRendered

%aimport gui.gui

pygame 2.1.2 (SDL 2.0.16, Python 3.9.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
genome = np.loadtxt('models/genetic-300limit-4.txt', dtype=float)[2]
genome

array([-2.65120500e+00, -1.40542180e+01,  9.17620500e+00, -4.85240000e-02,
        2.26107000e-01, -3.60218700e+00,  3.21621870e+01, -4.25700700e+00,
        9.95256800e+00, -4.02632527e+02, -4.54050280e+01, -1.02634300e+00,
        3.13316700e+00, -1.43648000e-01, -2.17000000e-04, -4.69923000e-01,
        1.38572000e-01, -5.04130000e-02, -4.54570000e-02, -7.49585000e-01,
        1.61800800e+00, -1.89829000e-01,  1.43494800e+00, -8.29420000e-01,
       -2.99942000e-01])

In [4]:
def get_best_state(heuristics, genome):
    weighted = np.multiply(heuristics, genome)
    
    # linear
    linear = np.sum(weighted, axis=1)
    # todo: test others
    
    return np.argmax(linear)

In [5]:
env = TetrisEnvRendered()

## Without hold

In [6]:
# runs = []
# def render_run():
#     env.reset()
#     states_to_render = []

#     while True:
#         states, states_pretty, scores, clears, heuristics, dones, moves = env.get_next_states()

#         chosen_index = get_best_state(heuristics, genome)

#         states_to_render.append((states_pretty[chosen_index], moves[chosen_index], env.current_piece, env.get_next_queue(), env.hold_piece, clears[chosen_index]))

#         if dones[chosen_index]:
#             print(f'Score: {env.score}')
#             print(f'Clears: {env.clears}, t-spins: {env.tspins}, alll_clears: {env.all_clears}')
#             break
#         else:
#             env.step(states[chosen_index], states_pretty[chosen_index], clears[chosen_index], scores[chosen_index])

#     runs.append(states_to_render)
    
# for i in range(1):
#     render_run()

## With hold

In [16]:
run = []
def render_run():
    env.reset()

    while True:
        states_curr, states_pretty_curr, scores_curr, clears_curr, heuristics_curr, dones_curr, moves_curr = env.get_next_states()
        states_hold, states_pretty_hold, scores_hold, clears_hold, heuristics_hold, dones_hold, moves_hold = env.get_next_states(use_hold=True)

        states = np.concatenate([states_curr, states_hold]) if states_hold is not None else states_curr
        states_pretty = np.concatenate([states_pretty_curr, states_pretty_hold]) if states_pretty_hold is not None else states_pretty_curr
        scores = scores_curr + scores_hold if scores_hold is not None else scores_curr
        clears = np.concatenate([clears_curr, clears_hold]) if clears_hold is not None else clears_curr
        heuristics = np.concatenate([heuristics_curr, heuristics_hold]) if heuristics_hold is not None else heuristics_curr
        dones = dones_curr + dones_hold if dones_hold is not None else dones_curr
        moves = np.concatenate([moves_curr, moves_hold]) if moves_hold is not None else moves_curr

        chosen_index = get_best_state(heuristics, genome)

        # it holded
        if chosen_index >= len(states_curr):
            run.append((states_pretty[chosen_index], moves[chosen_index], env.bag.peek_piece() if env.hold_piece is None else env.hold_piece, env.get_next_queue(), env.current_piece, clears[chosen_index]))
            env.hold()
        else:
            run.append((states_pretty[chosen_index], moves[chosen_index], env.current_piece, env.get_next_queue(), env.hold_piece, clears[chosen_index]))

        if env.moves % 1000 == 500:
            print(f'Score: {env.score}')
            print(f'Clears: {env.clears}, t-spins: {env.tspins}, alll_clears: {env.all_clears}')
            
        if dones[chosen_index]:
            print(f'Score: {env.score}')
            print(f'Clears: {env.clears}, t-spins: {env.tspins}, alll_clears: {env.all_clears}')
            break
        else:
            env.step(states[chosen_index], states_pretty[chosen_index], clears[chosen_index], scores[chosen_index])
    
render_run()

Score: 30500.0
Clears: [30, 25, 7, 23], t-spins: 0, alll_clears: 0
Score: 93212.5
Clears: [81, 61, 16, 76], t-spins: 0, alll_clears: 0


In [ ]:
# # store run
# with open('runs/530k.pkl', 'wb') as file:
#     pickle.dump(runs[0], file, pickle.HIGHEST_PROTOCOL)

# # load run
# with open('runs/396k.pkl', 'rb') as file:
#     run = pickle.load(file)

In [15]:
run2 = run.copy()

In [18]:
g = gui.gui.Gui(sleep=12)

last_state = np.ones((20,10)) * -1

time.sleep(10)

for state, moves, piece, queue, hold, clears in run: # runs[0][-200:]:
    p = Piece(piece)
    
    for move in moves:
        for m in move.split(','):
            if m == 'ml':
                p.pos[1] -= 1
            if m == 'mr':
                p.pos[1] += 1
            if m == 'sd':
                p.pos[0] += 1
            if m == 'mu':
                p.pos[0] -= 1

            if m == 'rr':
                p.rot = (p.rot + 1) % p.pdata.num_rot
            if m == 'rl':
                p.rot = (p.rot - 1) % p.pdata.num_rot

        g.draw(last_state, p, queue, hold)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            # if event.type == pygame.KEYDOWN:
            #     if event.key == pygame.K_RIGHT:
            #         move_step += 1
    
    if clears[0] == 4:
        g.play_sound_quad()
    elif clears[1] and clears[0] > 0:
        g.play_sound_tspin()
    elif clears[2]:
        pass # all-clear
    elif clears[0] > 0:
        g.play_sound_clear()
    
    last_state = state

In [19]:
pygame.quit()